# SQLAlchemy: Python Data Mapper ORM


In [97]:
import getpass
from urllib.parse import quote
from typing import Optional

In [98]:
db_user = getpass.getpass("Enter DB User")
db_password = quote(getpass.getpass("Enter DB Password"))
db_host = getpass.getpass("Enter DB Host")
db_name = "cms_python"

In [99]:
%pip install SQLAlchemy PyMySQL


Note: you may need to restart the kernel to use updated packages.


## SQL Alchemy Imports


In [100]:
from sqlalchemy import (
    Column,
    Date,
    ForeignKey,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    select,
)
from sqlalchemy.orm import (
    DeclarativeBase,
    Mapped,
    Session,
    aliased,
    mapped_column,
    relationship,
    sessionmaker,
)

## Initialize Engine with dialect and DB engine

- Ref:
  - ORM Quick Start: https://docs.sqlalchemy.org/en/20/orm/quickstart.html
  - Engine Guide: https://docs.sqlalchemy.org/en/20/core/engines.html


### **Do not Forget Exception Handling in Production**

In [101]:
db = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(
    db, echo=True
)  # echo=True will log background SQL operations

## list all tables and it's column


In [102]:
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(f"{table.name=}".center(50, "="))
    for column in table.c:
        print(f"{column.name=}")
    print()

2023-10-29 15:22:51,075 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-10-29 15:22:51,075 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,075 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `cms_python`
2023-10-29 15:22:51,080 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,091 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `comment`
2023-10-29 15:22:51,092 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,094 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `content`
2023-10-29 15:22:51,095 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,097 INFO sqlalchemy.engi

2023-10-29 15:22:51,181 INFO sqlalchemy.engine.Engine 
                    select table_schema, table_name, column_name
                    from information_schema.columns
                    where (table_schema, table_name, lower(column_name)) in
                    ((%(table_data_1_1)s, %(table_data_1_2)s, %(table_data_1_3)s));
                
2023-10-29 15:22:51,181 INFO sqlalchemy.engine.Engine [cached since 0.06594s ago] {'table_data_1_1': 'cms_python', 'table_data_1_2': 'user', 'table_data_1_3': 'user_id'}
2023-10-29 15:22:51,181 INFO sqlalchemy.engine.Engine 
                    select table_schema, table_name, column_name
                    from information_schema.columns
                    where (table_schema, table_name, lower(column_name)) in
                    ((%(table_data_1_1)s, %(table_data_1_2)s, %(table_data_1_3)s), (%(table_data_2_1)s, %(table_data_2_2)s, %(table_data_2_3)s));
                
2023-10-29 15:22:51,181 INFO sqlalchemy.engine.Engine [cached since 0.

## Session

- Ref:
  - Basic of Session: https://docs.sqlalchemy.org/en/20/orm/session_basics.html
  - Session API: https://docs.sqlalchemy.org/en/20/orm/session_api.html


In [103]:
session = Session(engine)  # for modular approach

# prefer contextual approach for functional programming
# with Session(engine) as session:
#     ...

In [104]:
# if there's only one Engine in module we can bind it
# Session = sessionmaker(bind=engine)

# session = Session()  # for modular approach

# prefer contextual approach for functional programming
# with Session() as session:
#     ...

## Declarative Class Base Approached

- Ref:
  - User guide: https://docs.sqlalchemy.org/en/20/orm/extensions/declarative/basic_use.html
  - API: https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase


In [105]:
class Base(DeclarativeBase):
    pass

- Ref:
  - Many to Many Guide: https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html#many-to-many


In [106]:
role_permission = Table(
    "role_permission",
    Base.metadata,
    Column("role_id", Integer, ForeignKey("role.role_id"), primary_key=True),
    Column(
        "permission_id",
        Integer,
        ForeignKey("permission.permission_id"),
        primary_key=True,
    ),
)


user_role = Table(
    "user_role",
    Base.metadata,
    Column(
        "user_id",
        Integer,
        ForeignKey(
            "user.user_id",
        ),
        primary_key=True,
    ),
    Column(
        "role_id",
        Integer,
        ForeignKey("role.role_id", ondelete="CASCADE"),
        primary_key=True,
    ),
)

In [107]:
class Role(Base):
    __tablename__ = "role"
    role_id: Mapped[int] = mapped_column(primary_key=True)
    role_name: Mapped[str] = mapped_column(String(150), unique=True)
    description: Mapped[str] = mapped_column(String(255))
    permissions: Mapped[list["Permission"]] = relationship(
        secondary=role_permission, back_populates="roles"
    )
    users: Mapped[list["User"]] = relationship(
        secondary=user_role, back_populates="roles"
    )

    def __repr__(self) -> str:
        return f"Role(id={self.role_id!r}, name={self.role_name!r})"

In [108]:
class Permission(Base):
    __tablename__ = "permission"
    permission_id: Mapped[int] = mapped_column(primary_key=True)
    permission_name: Mapped[str] = mapped_column(String(45), unique=True)
    description: Mapped[str] = mapped_column(String(255))
    roles: Mapped[list[Role]] = relationship(
        secondary=role_permission, back_populates="permissions"
    )

    def __repr__(self) -> str:
        return f"Permission(id={self.permission_id!r}, user={self.permission_name!r})"

In [109]:
class User(Base):
    __tablename__ = "user"
    # __table_args__ = {"extend_existing": True} # let you modify python object
    user_id: Mapped[int] = mapped_column(primary_key=True)
    user_name: Mapped[str] = mapped_column(String(45))
    email: Mapped[str] = mapped_column(String(255), unique=True)
    secret: Mapped["Secret"] = relationship(
        back_populates="user", cascade="all, delete", passive_deletes=True
    )
    roles: Mapped[list[Role]] = relationship(
        back_populates="users", secondary=user_role
    )
    contents: Mapped[list["Content"]] = relationship(
        back_populates="author", foreign_keys="Content.user_id"
    )

    def __repr__(self) -> str:
        return f"User(id={self.user_id!r}, name={self.user_name!r}, email={self.email!r})"

In [110]:
class Secret(Base):
    __tablename__ = "secret"
    secret_id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(
        ForeignKey("user.user_id", ondelete="CASCADE"), unique=True
    )
    password: Mapped[str] = mapped_column(String(45))
    expiry_date: Mapped[Date] = mapped_column(Date)

    user: Mapped[User] = relationship(back_populates="secret")

    def __repr__(self) -> str:
        return f"Secret(id={self.secret_id!r}, user={self.user_id!r})"

## Uncommon Join Condition

- Ref:
  - Join Conditions: https://docs.sqlalchemy.org/en/20/orm/join_conditions.html#handling-multiple-join-paths


In [111]:
class Content(Base):
    __tablename__ = "content"
    content_id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user.user_id"))
    contributor_id: Mapped[Optional[int]] = mapped_column(
        ForeignKey("user.user_id")
    )
    title: Mapped[str] = mapped_column(String(45))
    content: Mapped[str] = mapped_column(String(255))

    author: Mapped[User] = relationship(
        back_populates="contents", foreign_keys=user_id
    )
    comments: Mapped[list["Comment"]] = relationship("Comment")

    def __repr__(self) -> str:
        return f"Content(id={self.content_id!r}, title={self.title!r}, author={self.user_id!r})"

In [112]:
class Comment(Base):
    __tablename__ = "comment"
    comment_id: Mapped[int] = mapped_column(primary_key=True)
    content_id: Mapped[int] = mapped_column(ForeignKey("content.content_id"))
    user_id: Mapped[int] = mapped_column(ForeignKey("user.user_id"))
    comment: Mapped[str] = mapped_column(String(255))
    date: Mapped[Date] = mapped_column(Date)

    def __repr__(self) -> str:
        return f"Comment(id={self.comment_id!r}, content_id={self.content_id!r}, commenter={self.user_id!r}, date={self.date!r})"

In [113]:
# DO NOT DROP TABLES IN PRODUCTION
Base.metadata.drop_all(engine)

2023-10-29 15:22:51,457 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:51,459 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`role_permission`
2023-10-29 15:22:51,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,459 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`user_role`
2023-10-29 15:22:51,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,459 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`role`
2023-10-29 15:22:51,471 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,477 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`permission`
2023-10-29 15:22:51,481 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,481 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`user`
2023-10-29 15:22:51,481 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,488 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`secret`
2023-10-29 15:22:51,491 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-

In [114]:
Base.metadata.create_all(engine)

2023-10-29 15:22:51,670 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:51,674 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`role_permission`
2023-10-29 15:22:51,677 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,682 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`user_role`
2023-10-29 15:22:51,682 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,682 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`role`
2023-10-29 15:22:51,682 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`permission`
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`user`
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine DESCRIBE `cms_python`.`secret`
2023-10-29 15:22:51,690 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-

## Table: User & Secret

- Ref:
  - Basic Relation User Guide: https://docs.sqlalchemy.org/en/20/orm/basic_relationships.html
  - ORM Relation Object User Guide: https://docs.sqlalchemy.org/en/20/tutorial/orm_related_objects.html
  - Cascade Guide: https://docs.sqlalchemy.org/en/20/orm/cascades.html
  - Foreign Key API: https://docs.sqlalchemy.org/en/20/core/constraints.html#sqlalchemy.schema.ForeignKey
  - relationship API: https://docs.sqlalchemy.org/en/20/orm/relationship_api.html#relationships-api


In [115]:
users_data = [
    {
        "user_name": "Johnny",
        "email": "johnny@example.com",
        "secret": {"password": "Johnny_hashed_pw", "expiry_date": "2023-10-10"},
    },
    {
        "user_name": "Jane",
        "email": "jane@example.com",
        "secret": {"password": "Jane_hashed_pw", "expiry_date": "2023-10-15"},
    },
    {
        "user_name": "Bravo",
        "email": "bravo@example.com",
        "secret": {"password": "Bravo_hashed_pw", "expiry_date": "2023-10-11"},
    },
    {
        "user_name": "Doe",
        "email": "doe@example.com",
        "secret": {"password": "Doe_hashed_pw", "expiry_date": "2023-11-6"},
    },
    {
        "user_name": "Mary",
        "email": "mary@example.com",
        "secret": {"password": "Mary_hashed_pw", "expiry_date": "2023-11-20"},
    },
    {
        "user_name": "Lisa",
        "email": "lisa@example.com",
        "secret": {"password": "Lisa_hashed_pw", "expiry_date": "2023-12-05"},
    },
    {
        "user_name": "Raj",
        "email": "raj@example.com",
        "secret": {"password": "Raj_hashed_pw", "expiry_date": "2023-11-28"},
    },
    {
        "user_name": "Suraj",
        "email": "suraj@example.com",
        "secret": {"password": "Suraj_hashed_pw", "expiry_date": "2023-11-30"},
    },
    {
        "user_name": "Emma",
        "email": "emma@example.com",
        "secret": {"password": "Emma_hashed_pw", "expiry_date": "2023-12-15"},
    },
    {
        "user_name": "Ivan",
        "email": "ivan@example.com",
        "secret": {"password": "Ivan_hashed_pw", "expiry_date": "2023-12-6"},
    },
]

In [116]:
for user_data in users_data:
    secret = user_data.pop("secret")
    user = User(**user_data)
    secret = Secret(**secret)
    secret.user = user
    session.add_all([user, secret])

In [117]:
session.commit()

2023-10-29 15:22:52,055 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,055 INFO sqlalchemy.engine.Engine INSERT INTO user (user_name, email) VALUES (%(user_name)s, %(email)s)
2023-10-29 15:22:52,055 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {'user_name': 'Johnny', 'email': 'johnny@example.com'}
2023-10-29 15:22:52,075 INFO sqlalchemy.engine.Engine INSERT INTO user (user_name, email) VALUES (%(user_name)s, %(email)s)
2023-10-29 15:22:52,076 INFO sqlalchemy.engine.Engine [cached since 0.006568s ago] {'user_name': 'Jane', 'email': 'jane@example.com'}
2023-10-29 15:22:52,077 INFO sqlalchemy.engine.Engine INSERT INTO user (user_name, email) VALUES (%(user_name)s, %(email)s)
2023-10-29 15:22:52,078 INFO sqlalchemy.engine.Engine [cached since 0.009157s ago] {'user_name': 'Bravo', 'email': 'bravo@example.com'}
2023-10-29 15:22:52,080 INFO sqlalchemy.engine.Engine INSERT INTO user (user_name, email) VALUES (%(user_name)s, %(email)s)
2023-10-29 15:22:52,080 INFO s

## Select

- Ref:
  - ORM Query Guide: https://docs.sqlalchemy.org/en/20/orm/queryguide/index.html
  - ORM Select Guide: https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html
  - Session Query **(_Legacy as of 2.0_)** : https://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session.query


### User


In [118]:
for i in session.scalars(select(User)):  # generator
    print(i)

2023-10-29 15:22:52,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,159 INFO sqlalchemy.engine.Engine SELECT user.user_id, user.user_name, user.email 
FROM user
2023-10-29 15:22:52,160 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {}
User(id=1, name='Johnny', email='johnny@example.com')
User(id=2, name='Jane', email='jane@example.com')
User(id=3, name='Bravo', email='bravo@example.com')
User(id=4, name='Doe', email='doe@example.com')
User(id=5, name='Mary', email='mary@example.com')
User(id=6, name='Lisa', email='lisa@example.com')
User(id=7, name='Raj', email='raj@example.com')
User(id=8, name='Suraj', email='suraj@example.com')
User(id=9, name='Emma', email='emma@example.com')
User(id=10, name='Ivan', email='ivan@example.com')


In [119]:
session.execute(select(User)).all()  # returns row and not scalar object

2023-10-29 15:22:52,186 INFO sqlalchemy.engine.Engine SELECT user.user_id, user.user_name, user.email 
FROM user
2023-10-29 15:22:52,189 INFO sqlalchemy.engine.Engine [cached since 0.02922s ago] {}


[(User(id=1, name='Johnny', email='johnny@example.com'),),
 (User(id=2, name='Jane', email='jane@example.com'),),
 (User(id=3, name='Bravo', email='bravo@example.com'),),
 (User(id=4, name='Doe', email='doe@example.com'),),
 (User(id=5, name='Mary', email='mary@example.com'),),
 (User(id=6, name='Lisa', email='lisa@example.com'),),
 (User(id=7, name='Raj', email='raj@example.com'),),
 (User(id=8, name='Suraj', email='suraj@example.com'),),
 (User(id=9, name='Emma', email='emma@example.com'),),
 (User(id=10, name='Ivan', email='ivan@example.com'),)]

In [120]:
# LEGACY as of 2.0
session.query(User).all()  # list, list with scalars in Select Secret

2023-10-29 15:22:52,212 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.user_name AS user_user_name, user.email AS user_email 
FROM user
2023-10-29 15:22:52,213 INFO sqlalchemy.engine.Engine [generated in 0.00120s] {}


[User(id=1, name='Johnny', email='johnny@example.com'),
 User(id=2, name='Jane', email='jane@example.com'),
 User(id=3, name='Bravo', email='bravo@example.com'),
 User(id=4, name='Doe', email='doe@example.com'),
 User(id=5, name='Mary', email='mary@example.com'),
 User(id=6, name='Lisa', email='lisa@example.com'),
 User(id=7, name='Raj', email='raj@example.com'),
 User(id=8, name='Suraj', email='suraj@example.com'),
 User(id=9, name='Emma', email='emma@example.com'),
 User(id=10, name='Ivan', email='ivan@example.com')]

### Secret


In [121]:
session.scalars(
    select(Secret).where(Secret.expiry_date < "2023-11-01")  # filter or where
).all()  # list

2023-10-29 15:22:52,242 INFO sqlalchemy.engine.Engine SELECT secret.secret_id, secret.user_id, secret.password, secret.expiry_date 
FROM secret 
WHERE secret.expiry_date < %(expiry_date_1)s
2023-10-29 15:22:52,243 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {'expiry_date_1': '2023-11-01'}


[Secret(id=1, user=1), Secret(id=2, user=2), Secret(id=3, user=3)]

In [122]:
# LEGACY as of 2.0
session.query(Secret).filter(
    Secret.expiry_date < "2023-11-01"
).all()  # filter or where

2023-10-29 15:22:52,258 INFO sqlalchemy.engine.Engine SELECT secret.secret_id AS secret_secret_id, secret.user_id AS secret_user_id, secret.password AS secret_password, secret.expiry_date AS secret_expiry_date 
FROM secret 
WHERE secret.expiry_date < %(expiry_date_1)s
2023-10-29 15:22:52,259 INFO sqlalchemy.engine.Engine [generated in 0.00084s] {'expiry_date_1': '2023-11-01'}


[Secret(id=1, user=1), Secret(id=2, user=2), Secret(id=3, user=3)]

### Secret & User


In [123]:
query = (
    select(Secret, User)
    .join(Secret.user)
    .where(Secret.expiry_date < "2023-11-01")
)
print("".center(100, "-"))
print(f"{query=!s}")
print("".center(100, "-"))
for secret_user in session.scalars(
    query
):  # we are not doing extra query for user
    print(secret_user, secret_user.user)

----------------------------------------------------------------------------------------------------
query=SELECT secret.secret_id, secret.user_id, secret.password, secret.expiry_date, "user".user_id AS user_id_1, "user".user_name, "user".email 
FROM secret JOIN "user" ON "user".user_id = secret.user_id 
WHERE secret.expiry_date < :expiry_date_1
----------------------------------------------------------------------------------------------------
2023-10-29 15:22:52,278 INFO sqlalchemy.engine.Engine SELECT secret.secret_id, secret.user_id, secret.password, secret.expiry_date, user.user_id AS user_id_1, user.user_name, user.email 
FROM secret INNER JOIN user ON user.user_id = secret.user_id 
WHERE secret.expiry_date < %(expiry_date_1)s
2023-10-29 15:22:52,279 INFO sqlalchemy.engine.Engine [generated in 0.00096s] {'expiry_date_1': '2023-11-01'}
Secret(id=1, user=1) User(id=1, name='Johnny', email='johnny@example.com')
Secret(id=2, user=2) User(id=2, name='Jane', email='jane@example.com')
S

## Table: Role, Permission & Role_Permission

- Ref:
  - Core CRUD Tutorial: https://docs.sqlalchemy.org/en/20/tutorial/data.html
  - Core DML: https://docs.sqlalchemy.org/en/20/core/dml.html


## Bulk CRUD

- Ref:
  - ORM Bulk CRUD: https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html
  - ORM Single Update & Delete: https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria


In [124]:
session.execute(
    insert(Role),
    [
        {
            "role_name": "Administrator",
            "description": "Root role with all permissions",
        },
        {
            "role_name": "Editor",
            "description": "Publish, edit, view, comment and moderate comments",
        },
        {
            "role_name": "Author",
            "description": "Create articles, view and comment",
        },
        {
            "role_name": "Contributor",
            "description": "Contribute articles for other author, view and comment",
        },
        {"role_name": "Viewer", "description": "Able to view and comment"},
    ],
)

2023-10-29 15:22:52,295 INFO sqlalchemy.engine.Engine INSERT INTO `role` (role_name, description) VALUES (%(role_name)s, %(description)s)
2023-10-29 15:22:52,296 INFO sqlalchemy.engine.Engine [generated in 0.00090s] [{'role_name': 'Administrator', 'description': 'Root role with all permissions'}, {'role_name': 'Editor', 'description': 'Publish, edit, view, comment and moderate comments'}, {'role_name': 'Author', 'description': 'Create articles, view and comment'}, {'role_name': 'Contributor', 'description': 'Contribute articles for other author, view and comment'}, {'role_name': 'Viewer', 'description': 'Able to view and comment'}]


In [125]:
session.execute(
    insert(Permission),
    [
        {
            "permission_name": "MyContent",
            "description": "Create, edit and delete own contents",
        },
        {"permission_name": "Create", "description": "Create content"},
        {"permission_name": "Publish", "description": "Publish content"},
        {"permission_name": "Edit", "description": "Edit Content"},
        {"permission_name": "Delete", "description": "Delete Content"},
        {"permission_name": "View", "description": "View Content"},
        {
            "permission_name": "MyComment",
            "description": "Add, edit and delete own comments",
        },
        {
            "permission_name": "Moderate Comment",
            "description": "Edit or delete other's comments",
        },
        {
            "permission_name": "Manage User",
            "description": "Add or remove users",
        },
        {
            "permission_name": "Manage Role",
            "description": "Add, edit and remove roles",
        },
        {
            "permission_name": "Analytics",
            "description": "Access to analysis",
        },
    ],
)

2023-10-29 15:22:52,323 INFO sqlalchemy.engine.Engine INSERT INTO permission (permission_name, description) VALUES (%(permission_name)s, %(description)s)
2023-10-29 15:22:52,324 INFO sqlalchemy.engine.Engine [generated in 0.00109s] [{'permission_name': 'MyContent', 'description': 'Create, edit and delete own contents'}, {'permission_name': 'Create', 'description': 'Create content'}, {'permission_name': 'Publish', 'description': 'Publish content'}, {'permission_name': 'Edit', 'description': 'Edit Content'}, {'permission_name': 'Delete', 'description': 'Delete Content'}, {'permission_name': 'View', 'description': 'View Content'}, {'permission_name': 'MyComment', 'description': 'Add, edit and delete own comments'}, {'permission_name': 'Moderate Comment', 'description': "Edit or delete other's comments"}  ... displaying 10 of 11 total bound parameter sets ...  {'permission_name': 'Manage Role', 'description': 'Add, edit and remove roles'}, {'permission_name': 'Analytics', 'description': 'A

In [126]:
session.commit()

2023-10-29 15:22:52,337 INFO sqlalchemy.engine.Engine COMMIT


In [127]:
# in actual use case setting we would be referencing id from inserted object itself
# Here it's fixed mapping of role and permission and for blog purpose
role_permission_mapping = [
    {"role_id": 1, "permission_id": 1},
    {"role_id": 1, "permission_id": 2},
    {"role_id": 1, "permission_id": 3},
    {"role_id": 1, "permission_id": 4},
    {"role_id": 1, "permission_id": 5},
    {"role_id": 1, "permission_id": 6},
    {"role_id": 1, "permission_id": 7},
    {"role_id": 1, "permission_id": 8},
    {"role_id": 1, "permission_id": 9},
    {"role_id": 1, "permission_id": 10},
    {"role_id": 1, "permission_id": 11},
    {"role_id": 2, "permission_id": 3},
    {"role_id": 2, "permission_id": 4},
    {"role_id": 2, "permission_id": 6},
    {"role_id": 2, "permission_id": 7},
    {"role_id": 2, "permission_id": 8},
    {"role_id": 3, "permission_id": 1},
    {"role_id": 3, "permission_id": 6},
    {"role_id": 3, "permission_id": 7},
    {"role_id": 4, "permission_id": 2},
    {"role_id": 4, "permission_id": 6},
    {"role_id": 4, "permission_id": 7},
    {"role_id": 5, "permission_id": 6},
    {"role_id": 5, "permission_id": 7},
]

In [128]:
# manual approach of many-to-many relation
session.execute(role_permission.insert().values(role_permission_mapping))
session.commit()

2023-10-29 15:22:52,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,398 INFO sqlalchemy.engine.Engine INSERT INTO role_permission (role_id, permission_id) VALUES (%(role_id_m0)s, %(permission_id_m0)s), (%(role_id_m1)s, %(permission_id_m1)s), (%(role_id_m2)s, %(permission_id_m2)s), (%(role_id_m3)s, %(permission_id_m3)s), (%(role_id_m4)s, %(permission_id_m4)s), (%(role_id_m5)s, %(permission_id_m5)s), (%(role_id_m6)s, %(permission_id_m6)s), (%(role_id_m7)s, %(permission_id_m7)s), (%(role_id_m8)s, %(permission_id_m8)s), (%(role_id_m9)s, %(permission_id_m9)s), (%(role_id_m10)s, %(permission_id_m10)s), (%(role_id_m11)s, %(permission_id_m11)s), (%(role_id_m12)s, %(permission_id_m12)s), (%(role_id_m13)s, %(permission_id_m13)s), (%(role_id_m14)s, %(permission_id_m14)s), (%(role_id_m15)s, %(permission_id_m15)s), (%(role_id_m16)s, %(permission_id_m16)s), (%(role_id_m17)s, %(permission_id_m17)s), (%(role_id_m18)s, %(permission_id_m18)s), (%(role_id_m19)s, %(permission_id_m19

## Single Select


In [129]:
role_res = session.scalars(
    select(Role)
).all()  # not executing Join, i.e. associated data is not populated

print("Role Result".center(50, "="))
role_res

2023-10-29 15:22:52,425 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,427 INFO sqlalchemy.engine.Engine SELECT `role`.role_id, `role`.role_name, `role`.description 
FROM `role`
2023-10-29 15:22:52,428 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {}
===================Role Result====================


[Role(id=1, name='Administrator'),
 Role(id=2, name='Editor'),
 Role(id=3, name='Author'),
 Role(id=4, name='Contributor'),
 Role(id=5, name='Viewer')]

In [130]:
print("Role & Permission Result".center(100, "="))
# now we want permission also, we can iterate .permission property on role_res itself to fetch
for role in role_res:
    print(
        role, role.permissions
    )  # will execute explicit SELECT to fetch permission
    print()

======================================Role & Permission Result======================================
2023-10-29 15:22:52,445 INFO sqlalchemy.engine.Engine SELECT permission.permission_id AS permission_permission_id, permission.permission_name AS permission_permission_name, permission.description AS permission_description 
FROM permission, role_permission 
WHERE %(param_1)s = role_permission.role_id AND permission.permission_id = role_permission.permission_id
2023-10-29 15:22:52,446 INFO sqlalchemy.engine.Engine [generated in 0.00094s] {'param_1': 1}
Role(id=1, name='Administrator') [Permission(id=1, user='MyContent'), Permission(id=2, user='Create'), Permission(id=3, user='Publish'), Permission(id=4, user='Edit'), Permission(id=5, user='Delete'), Permission(id=6, user='View'), Permission(id=7, user='MyComment'), Permission(id=8, user='Moderate Comment'), Permission(id=9, user='Manage User'), Permission(id=10, user='Manage Role'), Permission(id=11, user='Analytics')]

2023-10-29 15:22:5

## Join Select


In [131]:
query = (
    select(Role, Permission)
    .select_from(role_permission)
    .join(Role)
    .join(Permission)
)
print("".center(100, "-"))
print(f"{query=!s}")
print("".center(100, "-"))
for role_permission_res in session.scalars(
    query
):  # we are not doing explicit SELECT query for permission
    print(role_permission_res, role_permission_res.permissions)

----------------------------------------------------------------------------------------------------
query=SELECT role.role_id, role.role_name, role.description, permission.permission_id, permission.permission_name, permission.description AS description_1 
FROM role_permission JOIN role ON role.role_id = role_permission.role_id JOIN permission ON permission.permission_id = role_permission.permission_id
----------------------------------------------------------------------------------------------------
2023-10-29 15:22:52,482 INFO sqlalchemy.engine.Engine SELECT `role`.role_id, `role`.role_name, `role`.description, permission.permission_id, permission.permission_name, permission.description AS description_1 
FROM role_permission INNER JOIN `role` ON `role`.role_id = role_permission.role_id INNER JOIN permission ON permission.permission_id = role_permission.permission_id
2023-10-29 15:22:52,482 INFO sqlalchemy.engine.Engine [generated in 0.00150s] {}
Role(id=1, name='Administrator') [Per

## Table: User_Role

- Ref:
  - Column Based Filter: https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.Column


In [132]:
role_user_mappings = {1: [1], 2: [2, 10], 3: [4, 5, 7], 4: [6, 8], 5: [3, 9]}

In [133]:
for role, users in role_user_mappings.items():
    role_res = session.scalar(select(Role).where(Role.role_id == role))
    role_res.users.extend(
        session.scalars(select(User).where(User.user_id.in_(users))).all()
    )  # Due to ORM relationships mapping, if we add/append in any one object it will be added in other object and conjunction table

2023-10-29 15:22:52,526 INFO sqlalchemy.engine.Engine SELECT `role`.role_id, `role`.role_name, `role`.description 
FROM `role` 
WHERE `role`.role_id = %(role_id_1)s
2023-10-29 15:22:52,527 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {'role_id_1': 1}
2023-10-29 15:22:52,530 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.user_name AS user_user_name, user.email AS user_email 
FROM user, user_role 
WHERE %(param_1)s = user_role.role_id AND user.user_id = user_role.user_id
2023-10-29 15:22:52,530 INFO sqlalchemy.engine.Engine [generated in 0.00072s] {'param_1': 1}
2023-10-29 15:22:52,534 INFO sqlalchemy.engine.Engine SELECT user.user_id, user.user_name, user.email 
FROM user 
WHERE user.user_id IN (%(user_id_1_1)s)
2023-10-29 15:22:52,535 INFO sqlalchemy.engine.Engine [generated in 0.00089s] {'user_id_1_1': 1}
2023-10-29 15:22:52,538 INFO sqlalchemy.engine.Engine INSERT INTO user_role (user_id, role_id) VALUES (%(user_id)s, %(role_id)s)
2023-10-29 15:22:52

In [134]:
session.commit()

2023-10-29 15:22:52,609 INFO sqlalchemy.engine.Engine INSERT INTO user_role (user_id, role_id) VALUES (%(user_id)s, %(role_id)s)
2023-10-29 15:22:52,610 INFO sqlalchemy.engine.Engine [cached since 0.05939s ago] [{'user_id': 3, 'role_id': 5}, {'user_id': 9, 'role_id': 5}]
2023-10-29 15:22:52,612 INFO sqlalchemy.engine.Engine COMMIT


## Table: Content


In [135]:
content_detail = [
    {
        "user_id": 4,
        "title": "RDBMS Relation",
        "content": "Relation between DBMS entities",
    },
    {
        "user_id": 5,
        "title": "Phoenix",
        "content": "Rising from the ash.",
    },
    {
        "user_id": 4,
        "title": "Git",
        "content": "Git is SCM(Source Control Management)",
    },
    {
        "user_id": 5,
        "contributor_id": 6,
        "title": "Shell on Rabbit",
        "content": "OS independent Shell customization",
    },
    {
        "user_id": 7,
        "contributor_id": 8,
        "title": "NoSQL",
        "content": "Concealed Schema DBMS",
    },
    {
        "user_id": 5,
        "contributor_id": 8,
        "title": "Docker",
        "content": "Developing, shipping and running containers",
    },
]

In [136]:
for content_data in content_detail:
    content = Content(**content_data)
    session.add(content)

In [137]:
session.commit()

2023-10-29 15:22:52,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,677 INFO sqlalchemy.engine.Engine INSERT INTO content (user_id, contributor_id, title, content) VALUES (%(user_id)s, %(contributor_id)s, %(title)s, %(content)s)
2023-10-29 15:22:52,678 INFO sqlalchemy.engine.Engine [generated in 0.00122s] {'user_id': 4, 'contributor_id': None, 'title': 'RDBMS Relation', 'content': 'Relation between DBMS entities'}
2023-10-29 15:22:52,681 INFO sqlalchemy.engine.Engine INSERT INTO content (user_id, contributor_id, title, content) VALUES (%(user_id)s, %(contributor_id)s, %(title)s, %(content)s)
2023-10-29 15:22:52,681 INFO sqlalchemy.engine.Engine [cached since 0.004577s ago] {'user_id': 5, 'contributor_id': None, 'title': 'Phoenix', 'content': 'Rising from the ash.'}
2023-10-29 15:22:52,683 INFO sqlalchemy.engine.Engine INSERT INTO content (user_id, contributor_id, title, content) VALUES (%(user_id)s, %(contributor_id)s, %(title)s, %(content)s)
2023-10-29 15:22:52,

## Alias

- Ref:
  - ORM Alias: https://docs.sqlalchemy.org/en/20/tutorial/data_select.html#orm-entity-aliases


In [139]:
user_author = aliased(User)
for content in session.scalars(
    select(Content)
    .join(user_author, Content.user_id == user_author.user_id)
    .join(User, Content.contributor_id == User.user_id, isouter=True)
):
    print(content, content.author)
    print()

2023-10-29 15:22:52,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,733 INFO sqlalchemy.engine.Engine SELECT content.content_id, content.user_id, content.contributor_id, content.title, content.content 
FROM content INNER JOIN user AS user_1 ON content.user_id = user_1.user_id LEFT OUTER JOIN user ON content.contributor_id = user.user_id
2023-10-29 15:22:52,733 INFO sqlalchemy.engine.Engine [generated in 0.00064s] {}
2023-10-29 15:22:52,737 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.user_name AS user_user_name, user.email AS user_email 
FROM user 
WHERE user.user_id = %(pk_1)s
2023-10-29 15:22:52,738 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {'pk_1': 4}
Content(id=1, title='RDBMS Relation', author=4) User(id=4, name='Doe', email='doe@example.com')

2023-10-29 15:22:52,742 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.user_name AS user_user_name, user.email AS user_email 
FROM user 
WHERE user.user_

## Table: Comment


In [140]:
comment_details = [
    {
        "content_id": 1,
        "user_id": 1,
        "comment": "Nice Pick",
        "date": "2023-10-03",
    },
    {
        "content_id": 1,
        "user_id": 4,
        "comment": "Thank You",
        "date": "2023-10-03",
    },
    {
        "content_id": 3,
        "user_id": 9,
        "comment": "Git is surely essential",
        "date": "2023-10-15",
    },
    {
        "content_id": 6,
        "user_id": 6,
        "comment": "To the point contributed by Suraj",
        "date": "2023-10-23",
    },
    {
        "content_id": 6,
        "user_id": 6,
        "comment": "Wish to keep it small and concise",
        "date": "2023-10-24",
    },
    {
        "content_id": 2,
        "user_id": 5,
        "comment": "Like, share and follow for more",
        "date": "2023-10-25",
    },
]

In [141]:
for comment_data in comment_details:
    comment = Comment(**comment_data)
    session.add(comment)

In [142]:
session.commit()

2023-10-29 15:22:52,791 INFO sqlalchemy.engine.Engine INSERT INTO comment (content_id, user_id, comment, date) VALUES (%(content_id)s, %(user_id)s, %(comment)s, %(date)s)
2023-10-29 15:22:52,791 INFO sqlalchemy.engine.Engine [generated in 0.00072s] {'content_id': 1, 'user_id': 1, 'comment': 'Nice Pick', 'date': '2023-10-03'}
2023-10-29 15:22:52,794 INFO sqlalchemy.engine.Engine INSERT INTO comment (content_id, user_id, comment, date) VALUES (%(content_id)s, %(user_id)s, %(comment)s, %(date)s)
2023-10-29 15:22:52,794 INFO sqlalchemy.engine.Engine [cached since 0.003769s ago] {'content_id': 1, 'user_id': 4, 'comment': 'Thank You', 'date': '2023-10-03'}
2023-10-29 15:22:52,796 INFO sqlalchemy.engine.Engine INSERT INTO comment (content_id, user_id, comment, date) VALUES (%(content_id)s, %(user_id)s, %(comment)s, %(date)s)
2023-10-29 15:22:52,797 INFO sqlalchemy.engine.Engine [cached since 0.006184s ago] {'content_id': 3, 'user_id': 9, 'comment': 'Git is surely essential', 'date': '2023-10-

## Order By Select

- Ref:
  - Selectable: https://docs.sqlalchemy.org/en/20/core/selectable.html


In [143]:
session.scalars(select(Comment).order_by(Comment.content_id)).all()

2023-10-29 15:22:52,834 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-29 15:22:52,835 INFO sqlalchemy.engine.Engine SELECT comment.comment_id, comment.content_id, comment.user_id, comment.comment, comment.date 
FROM comment ORDER BY comment.content_id
2023-10-29 15:22:52,836 INFO sqlalchemy.engine.Engine [generated in 0.00072s] {}


[Comment(id=1, content_id=1, commenter=1, date=datetime.date(2023, 10, 3)),
 Comment(id=2, content_id=1, commenter=4, date=datetime.date(2023, 10, 3)),
 Comment(id=6, content_id=2, commenter=5, date=datetime.date(2023, 10, 25)),
 Comment(id=3, content_id=3, commenter=9, date=datetime.date(2023, 10, 15)),
 Comment(id=4, content_id=6, commenter=6, date=datetime.date(2023, 10, 23)),
 Comment(id=5, content_id=6, commenter=6, date=datetime.date(2023, 10, 24))]

In [144]:
for content in session.scalars(select(Content)):
    print(content)
    print(content.comments)  # will make explicit SELECT query
    print()

2023-10-29 15:22:52,865 INFO sqlalchemy.engine.Engine SELECT content.content_id, content.user_id, content.contributor_id, content.title, content.content 
FROM content
2023-10-29 15:22:52,866 INFO sqlalchemy.engine.Engine [generated in 0.00096s] {}
Content(id=1, title='RDBMS Relation', author=4)
2023-10-29 15:22:52,870 INFO sqlalchemy.engine.Engine SELECT comment.comment_id AS comment_comment_id, comment.content_id AS comment_content_id, comment.user_id AS comment_user_id, comment.comment AS comment_comment, comment.date AS comment_date 
FROM comment 
WHERE %(param_1)s = comment.content_id
2023-10-29 15:22:52,871 INFO sqlalchemy.engine.Engine [generated in 0.00119s] {'param_1': 1}
[Comment(id=1, content_id=1, commenter=1, date=datetime.date(2023, 10, 3)), Comment(id=2, content_id=1, commenter=4, date=datetime.date(2023, 10, 3))]

Content(id=2, title='Phoenix', author=5)
2023-10-29 15:22:52,874 INFO sqlalchemy.engine.Engine SELECT comment.comment_id AS comment_comment_id, comment.content

In [145]:
session.close()

2023-10-29 15:22:52,912 INFO sqlalchemy.engine.Engine ROLLBACK


## Reach Out

For more intuitive blogs follow me on Medium & Github. You can also reach out to me via LinkedIn or X(Twitter).

[![Shivam Panchal | LinkedIn](https://img.shields.io/badge/Shivam_Panchal-eeeeee?style=for-the-badge&logo=linkedin&logoColor=ffffff&labelColor=0A66C2)][reach_linkedin]
[![l_shivam_l | X](https://img.shields.io/badge/l__shivam__l-eeeeee?style=for-the-badge&logo=x&logoColor=ffffff&labelColor=000000)][reach_x]
[![GodWin1100 | GitHub](https://img.shields.io/badge/Godwin1100-eeeeee?style=for-the-badge&logo=github&logoColor=ffffff&labelColor=181717)][reach_github]
[![GodWin | Medium](https://img.shields.io/badge/Shivam_Panchal-eeeeee?style=for-the-badge&logo=medium&logoColor=ffffff&labelColor=000000)][reach_medium]

[reach_linkedin]: https://www.linkedin.com/in/godwin1100
[reach_x]: https://twitter.com/l_shivam_l
[reach_medium]: https://medium.com/@godwin1100
[reach_github]: https://github.com/GodWin1100
